vamos a morirnos

In [7]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
from pymongo import MongoClient
from pandas import json_normalize
from pymongo import MongoClient,GEOSPHERE
import shapely.geometry
import geopandas as gdp
from bson.json_util import dumps

In [13]:
import src.conversion_functions as cf 
import src.geofunctions as gf

In [6]:
df = pd.read_csv('newyork_data.csv', index_col = 0)
df2 = pd.read_csv('mad_data.csv', index_col = 0)
df3 = pd.read_csv('bcn_data.csv', index_col = 0)

In [18]:
client = MongoClient("localhost:27017")
db = client.get_database("GeoProject3")

In [21]:
m = db.get_collection("mad")
b = db.get_collection("bcn")
ny = db.get_collection('ny')

Empezamos con las geoqueries

Las distancias de 'distance' no tienen sentido en Madrid y Barcelona, en Nueva York coinciden y tienen snetido (comprobado), sin embargo tanto en Madrid como en Barcelona no (comprobado también, respecto a las coordenadas impuestas por mí). 
Por ello pasamos a utilizar la columna de distancia que se generó directamente en el primer json en vez de la de distance que hemos generado con el geoNear.

(Es mentira  que desde donde he puesto las coordenadas de Madrid, hasta el resturante vegano 'La huella Vegana' hay 500m, yo vivo al lado de este restaurante y hay bastnates kilómetros desde Las Rozas hasta la zona de 'La Vela' del BBVA).

In [22]:
ny_ = gf.todo2 ([-73.98996, 40.75673], ny)
bcn_ = gf.todo2([-3.67097, 40.50222], b)

In [14]:
mad_ = gf.todo2([2.19763, 41.40080], m)
mad_.head()

,nombre,ciudad,distancia,tipo
9,"Edificio Veganova, Alcobendas",Madrid,5476,Vegetarian / Vegan Restaurant
10,Bar y Restaurante Payger,Alcobendas,5676,Restaurant
11,Bar Sube y Baja,Alcobendas,5293,Café
13,Starbucks Diversia Alcobendas,Alcobendas,4172,Cafeteria
15,Eventos Fly Up!,Madrid,4731,University


Ahora ya tenemos en 3 dataframes diferentes cada una de las ciudades con  nuestras 5 prioridades, no más lejos de 6 kilómetros.

Lo que tenemos que hacer ahora, es agrupar cada una de estas prioridades y sacar la media dela distancia de las mismas, seguidamente se ponderará cada una por el valor escogido según la prioridad que tenga frente a las otras. Tendremos un valor por cada una de las 5 prioridades estudiadas y por cada ciudad, por ello tendremos que realizar otra operación para quedarnos solo con un número por ciudad, el cual será el indicativo de cuan buena es esa ciudad según las características que hemo impuesto para la apertura de la empresa.

Vamos a hacer un groupby en cada dataframe por cada tipo diferente (para los Veganos, Startups, Veterinarios....)

In [54]:
ny_agregado = gf.agrega(ny_)
ny_agregado.head()

,tipo,distancia
0,American Restaurant,406.333333
1,Art Gallery,566.500000
2,Arts & Crafts Store,521.000000
3,Asian Restaurant,446.000000
4,BBQ Joint,544.000000


NY

Vegetarian / Vegan Restaurant = Vegan

Tech Startup = Startup

Bar = Bar

Café = Starbuks

Pet Service = Veterinarian

In [55]:
mad_agregado = gf.agrega(mad_)
mad_agregado.head()

,tipo,distancia
0,Bakery,5258.000000
1,Bar,2325.857143
2,Cafeteria,3047.500000
3,Café,4991.000000
4,Chinese Restaurant,445.000000


MAD

Vegetarian / Vegan Restaurant = Vegan

Bar = Bar

Cafeteria = Starbucks


In [76]:
bcn_agregado = gf.agrega(bcn_)
bcn_agregado.head()

,tipo,distancia
0,Accessories Store,3210.5
1,Arts & Crafts Store,2389.0
2,Athletics & Sports,4540.0
3,Board Shop,2959.0
4,Building,1683.5


BCN

Vegetarian / Vegan restaurant = Vegan

Café = Starbucks

Tech Startup = Startup

Food & Drink Shop = Bar


## Primeras observaciones

Tan solo en Nueva York encontramos las 5 prioridades dentro un radio de 6 kilómetros y en Madrid solo tenemos 3 de las 5 prioridades.
De primeras vamos a eliminar Madrid de nuestras posibles ciudades.
El veterinario vamos a dejarlo en un segundo plano, aun sabiendo que sólo en Nuevea York hay a una distancia menor de 6km.

Nos quedamos solo con los tipos que nos interesan :

In [77]:
ny_acortado = ny_agregado[(ny_agregado['tipo'] == 'Vegetarian / Vegan Restaurant') | (ny_agregado['tipo'] == 'Café') | (ny_agregado['tipo'] == 'Tech Startup') | (ny_agregado['tipo'] == 'Bar')]

In [78]:
ny_acortado

,tipo,distancia
6,Bar,873.923077
12,Café,695.500000
54,Tech Startup,1475.000000
56,Vegetarian / Vegan Restaurant,2913.526316


In [79]:
bcn_acortado = bcn_agregado[(bcn_agregado['tipo'] == 'Vegetarian / Vegan Restaurant') | (bcn_agregado['tipo'] == 'Café') | (bcn_agregado['tipo'] == 'Tech Startup') | (bcn_agregado['tipo'] == 'Food & Drink Shop')]

In [81]:
bcn_acortado

,tipo,distancia
5,Café,2605.500000
16,Food & Drink Shop,2692.000000
29,Tech Startup,1632.666667
30,Vegetarian / Vegan Restaurant,2834.571429


## Ponderizamos

Ahora por cada prioridad vamos a ponderizar de la siguiente manera:

- Start up x 1.0

- Vegano x 0.85

- Starbucks x 0.75

- Bar x 0.25

(recordamos que NY es la única ciudad con veterinario cerca, pero para no descartar también Barcelona, vamos a dejar el veterinario en un segundo plano)

### Ponderización de Nueva York

- Start up = 1475 x 1 = 1475
- Vegano = 2913.52 x 0.85 = 2476.5
- Starbucks = 695.5 x 0.75 = 521.63
- Bar = 873.92 x 0.25 = 218.48

#### Normalización y puntuación final de Nueva York

4.69

### Ponderización de Barcelona

- Start Up = 1632.66 x 1 = 1632.66
- Vegano = 2834.57 x 0.85 = 2409.38
- Starbucks = 2605.5 x 0.75 = 1954.13
- Bar = 2692 x 0.25 = 673

#### Normalización y puntuación final de Barcelona

6.67

## Resultados y conclusiones

Estos valores últimos de de normalización y puntuación se han sacado de la siguiente manera:
- Se han sumado todos los valores de cada ciudad y nos da un reultado del orden de maginitud de miles
- Se divide este sumatorio entre 1000 para tener una normalización sobre 10
- El mejor valor será el más bajo, esto nos indica que de forma general, las condiciones de la empresa se encuentran más cerca de nuestras coordenadas 

Por lo tanto, ya que 4.69 < 6.67, Nueva York debería ser nuestra ciudad donde instalemos la empresa, ya que de una manera más general, tenemos los requerimientos más cerca que Barcelona y Madrid.
Además, acordemonos de que Nueva York es la única ciudad con un veterinario en un radio menor de 6 kilómetros, por lo tanto un plus añadido.